# Import di tutte le librerie necessarie, inclusa la nostra libreria di supporto

In [1]:
# Nostre librerie
from ts_train.step.aggregation import Aggregation
from ts_train.step.filling import Filling
from ts_train.step.time_bucketing import TimeBucketing


# Librerie terze
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import random
import plotly.graph_objects as go
from plotly.offline import iplot
from pyspark.sql import functions as F
import plotly.graph_objects as go
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import count
from pyspark.sql.functions import col, lit

# Codice per visualizzazione su notebook
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Metodi secondari utili solo per questa demo

In [2]:
def get_random_user(filled_df):
    all_user_ids = filled_df.select('ID_CLIENTE_BIC').distinct().collect()
    all_user_ids = [row.ID_CLIENTE_BIC for row in all_user_ids]
    rand_id = random.randint(0, len(all_user_ids)-1)
    rand_user_id = all_user_ids[rand_id]
    return rand_user_id


def plot(x_values, y_values, x_name, y_name, title, mode="lines"):
    # Create the time series line plot with Plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_values, y=y_values, mode=mode, name=title))
    fig.update_layout(title=title,
                      xaxis_title=x_name,
                      yaxis_title=y_name,
                      showlegend=True,
                      template='plotly_white')
    
    # Show the plot directly in the Jupyter Notebook
    iplot(fig)

def get_values(df,col_name):
    return df.select(col(col_name)).rdd.flatMap(lambda x: x).collect()




def filled(df,time_bucket_col_name,identifier_cols_name,time_bucket_size,time_bucket_granularity,new_timestamp_col_name,time_zone):
    # Creates a list of identifier columns
    identifier_cols = [
        F.col(identifier_col_name)
        for identifier_col_name in identifier_cols_name
    ]
    
    # Creates the bucket size
    time_bucket_duration = (
        str(time_bucket_size) + " " + time_bucket_granularity
    )
    
    # Creates aliases for simplicity and code readability
    time_bucket_start = f"{time_bucket_col_name}_start"
    time_bucket_end = f"{time_bucket_col_name}_end"
    min_time_bucket_start = f"min_{time_bucket_col_name}_start"
    max_time_bucket_end = f"max_{time_bucket_col_name}_end"
    
    # Creates a new DataFrame with only the identifier columns
    # Splits the bucket into two column, start and end assigning to new columns
    ids_df = df.select(
        *identifier_cols,
        F.col(time_bucket_col_name).start.alias(time_bucket_start),
        F.col(time_bucket_col_name).end.alias(time_bucket_end),
    )
    
    ids_df = ids_df.withColumn("bucket_end", F.date_trunc(time_bucket_granularity, "bucket_end"))
    ids_df = ids_df.withColumn("bucket_start", F.date_trunc(time_bucket_granularity, "bucket_start"))

    
    
    # Takes only one record for every user
    # Saves only the min start and the max end
    ids_df = ids_df.groupBy(*identifier_cols).agg(
        F.min(time_bucket_start).alias(min_time_bucket_start),
        F.max(time_bucket_end).alias(max_time_bucket_end),
    )
    
    
    # Creates a new column with inside for each user an array of timestamps from
    # the min to the max of the time bucket of that particular user
    # Drops min and max columns
    ids_timestamps_df = ids_df.withColumn(
        "timestamps",
        F.expr(
            f"sequence(to_timestamp({min_time_bucket_start}),"
            f" to_timestamp({max_time_bucket_end}), interval"
            f" {time_bucket_duration})"
        ),
    ).drop(
        min_time_bucket_start,
        max_time_bucket_end,
    )
    
    # Explodes the array of timestamps into a series of rows each with a timestamp
    # column representing the start of that time bucket
    # Drops timestamps array column
    ids_timestamps_df = ids_timestamps_df.withColumn(
        new_timestamp_col_name, F.explode(F.col("timestamps"))
    ).drop(
        "timestamps",
    )
    
    df = df.withColumn(
        new_timestamp_col_name, F.col(time_bucket_col_name).start
    )

    # Joins the DataFrame with the new DataFrame in which has been generated
    # timestamps for every user from its min timestamp to his max
    # Fills with 0 null values of every column
    # Drops time bucket column
    join_on_cols = [*identifier_cols_name, new_timestamp_col_name]
    df = df.withColumn(new_timestamp_col_name, F.date_trunc(time_bucket_granularity, new_timestamp_col_name))
    df = (
        df.join(ids_timestamps_df, on=join_on_cols, how="right")
        .fillna(0)  # TODO verify this has no negative effect
        .drop(time_bucket_col_name)  # TODO choose if we want to drop
    )

    df = df.orderBy(*identifier_cols_name, new_timestamp_col_name)

    return df 

def timerange_df(df,time_col_name):
    # Converte le date in formato "yyyy-MM-dd" in tipo DateType
    start_date = spark.sql("SELECT CAST('{}' AS DATE)".format(START_DATE)).first()[0]
    end_date = spark.sql("SELECT CAST('{}' AS DATE)".format(END_DATE)).first()[0]
    
    # Filtra le righe del DataFrame all'interno del range di date specificato
    df_plot = df.filter((col(time_col_name) >= start_date) & (col(time_col_name) <= end_date))
    return df_plot

def plot_in_timerange(df, time_col_name, y_col_name,title, mode):
    df_plot = timerange_df(df,time_col_name)
    
    x_values = get_values(df_plot, time_col_name)
    y_values = get_values(df_plot, y_col_name)
    
    # Call the plot function with the extracted x_values and y_values
    plot(x_values, y_values, x_name='Date', y_name=y_col_name, title=title, mode=mode)
    
def plot_timerange_buckets(time_bucket_df, time_col_name):

    # Extract the start of each bucket as x_values list and the counter for each bucket as y_values list
    x_values = get_values(time_bucket_df,time_col_name)
    y_values = get_values(time_bucket_df,"count") 
    
    
    # Create a bar plot
    fig = go.Figure()
    fig.add_trace(go.Bar(x=x_values, y=y_values))
    
    # Update layout
    fig.update_layout(title='Usefulness of Time Buckets',
                      xaxis_title='Timestamps in Bucket',
                      yaxis_title='Counter in Bucket',
                      template='plotly_white')
    
    # Show the plot
    fig.show()

In [3]:
# Variabili globali usate solo per la presentazione:

## si è deciso di mostrare un cliente specifico per avere dei plot più leggibili. L'utente è stato scelto a caso
## si è deciso di concentrarsi su un intervallo temporale ristretto per poter mostrare meglio i risultati sul plot.

DATA_COLUMN_NAME = "DATA_TRANSAZIONE"
START_DATE = "2023-02-15"
END_DATE = "2023-04-25"

# Presentazione del problema
Use case immaginato: predizione di abbandono carta in favore dei contanti


# Dataset Utilizzato:

- **Tabella di partenza**: cust_know.ck_trans_cat
- **Colonne selezionate**: ARCA_TIPO_CARTA, DATA_CONTABILE, DATA_TRANSAZIONE, ORA_TRANSAZIONE, IMPORTO,SEGNO, ID_CLIENTE_BIC, IS_CARTA, TIPO_CANALE, TIPO_CANALE_AGG, IS_BON, IS_SDD, CATEGORY_LIV0, CATEGORY_LIV1, CATEGORY_LIV2, IS_CC, IS_LIB, MERCHANT
- **Utenti estratti**: 1000
- **Totale transazioni**: 960.404


In [4]:
# LOADING DATA

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Read CSV") \
    .getOrCreate()

PATH_TO_DATA = "../../data/df_ts.csv"
original_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", "|") \
    .option("encoding", "utf-8") \
    .option("multiline", "False") \
    .csv(PATH_TO_DATA)


original_df = original_df.orderBy(col("ID_CLIENTE_BIC"),col(DATA_COLUMN_NAME)) # order by date


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/01 16:39:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Vediamo un estratto del dataset

Per comodità espositiva ci concetriamo solo su un utente

In [5]:
# Per comodità espositiva ci concetriamo solo su un utente
ID_CLIENTE = "89761829"

original_df = original_df.filter(F.col("ID_CLIENTE_BIC").isin(ID_CLIENTE)) 
original_df.show(truncate=False)

+---------------+--------------+----------------+-------------------+-------+-----+--------------+--------+--------------+---------------+------+------+-----------------+-----------------------------+-------------+-----+------+-----------+
|ARCA_TIPO_CARTA|DATA_CONTABILE|DATA_TRANSAZIONE|ORA_TRANSAZIONE    |IMPORTO|SEGNO|ID_CLIENTE_BIC|IS_CARTA|TIPO_CANALE   |TIPO_CANALE_AGG|IS_BON|IS_SDD|CATEGORY_LIV0    |CATEGORY_LIV1                |CATEGORY_LIV2|IS_CC|IS_LIB|MERCHANT   |
+---------------+--------------+----------------+-------------------+-------+-----+--------------+--------+--------------+---------------+------+------+-----------------+-----------------------------+-------------+-----+------+-----------+
|EVOLUTION      |2021-05-04    |2021-04-29      |2023-08-01 17:34:11|70.0   |-    |89761829      |true    |ATM_BANCARIO  |FISICO_ESTERNO |false |false |prelievo_contante|null                         |null         |false|false |null       |
|EVOLUTION      |2021-05-04    |2021-04-

# Iniziamo ad esplorare i dati

Plottiamo tutti gli importi dell' utente

In [6]:
# Extract the 'DATA_CONTABILE' and 'IMPORTO' columns and collect them as lists
x_values = get_values(original_df, DATA_COLUMN_NAME)
y_values = get_values(original_df, "IMPORTO")

# Call the plot function with the extracted x_values and y_values
plot(x_values, y_values, x_name='Date', y_name='Importo', title='Time Series of Importo')

Gli importi plottati della timeseries sono irregolare ed è molto difficile fare un qualsiasi tipo di analisi su questi dati.  
Facciamo uno zoom-in per capirne i motivi

In [7]:
plot_in_timerange(original_df,DATA_COLUMN_NAME,"IMPORTO","ZoomIN Timeseries importi",mode="lines")

### Presenza di eventi coincidenti, ovvero che si verificano nello stesso momento 


In [8]:
plot_in_timerange(original_df,DATA_COLUMN_NAME,"IMPORTO","ZoomIN Timeseries importi", mode='markers')

### Gli importi da soli sono poco significativi


In [9]:
plot_in_timerange(original_df,DATA_COLUMN_NAME,"IMPORTO","ZoomIN Timeseries importi", mode='markers')

### Assenza di eventi, ovvero vi sono dei "salti" nella linea temporale.

In [10]:
plot_in_timerange(original_df,DATA_COLUMN_NAME,"IMPORTO","ZoomIN Timeseries importi", mode='markers')

23/08/01 16:39:34 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


# Soluzione problema 1: 
Individuazione degli eventi coincidenza e raccolta degli stessi in time-buckets comuni


In [11]:
#### APPLICO IL TIME BUCKETING
time_bucket_step =  TimeBucketing(
        time_zone="Europe/Rome",
        time_column_name=DATA_COLUMN_NAME,
        time_bucket_size=1,
        time_bucket_granularity="day",
        time_bucket_col_name="bucket",
    )

time_bucket_df = time_bucket_step(original_df, spark)


In [12]:
timerange_bucket_df = timerange_df(time_bucket_df,DATA_COLUMN_NAME)

timerange_bucket_counter_df = timerange_bucket_df.groupBy("ID_CLIENTE_BIC", "bucket").agg(count("*").alias("count"))
timerange_bucket_counter_df = timerange_bucket_counter_df.withColumn("bucket_start", col("bucket").getItem("end"))
timerange_bucket_counter_df = timerange_bucket_counter_df.orderBy(col("bucket_start"))

In [13]:
#### PRINT ORIGINALE
plot_in_timerange(original_df,DATA_COLUMN_NAME,"IMPORTO","ZoomIN Timeseries importi", mode='markers')
ranged_original_df = timerange_df(original_df,DATA_COLUMN_NAME)
#ranged_original_df.show(truncate=False)

In [14]:
### PRINT TIMEBUCKET
plot_timerange_buckets(timerange_bucket_counter_df,"bucket_start")

# 3 Problemi principali:

- Presenza di eventi coincidenti, ovvero che si verificano nello stesso momento -> Time-Bucketing Step
- Gli importi da soli sono poco significativi --> XXX
- Assenza di eventi, ovvero vi sono dei "salti" nella linea temporale.

Soluzione problema 2: 
Aggregiazione di variabili categoriche/numeriche per la creazione di valori significativi

**Aggregazione 1**: 
Il totale dei soldi ritirati allo sportello: 

**SUM** di **IMPORTO** con
* **CATEGORY_LIV0** = prelievo_contante
* **SEGNO** = "+"

In [15]:
#### AGGREGATION_1: evoluzione dei prelievi nel tempo

prelievo_contanti = [("CATEGORY_LIV0", ["prelievo_contante"]), ("SEGNO", ["-"])]


aggregation_step =  Aggregation(
    numerical_col_name=["IMPORTO"],
    identifier_cols_name=["ID_CLIENTE_BIC"],
    all_aggregation_filters=[prelievo_contanti],
    agg_funcs=["sum"],
)

prelievi_aggregated_df = aggregation_step(time_bucket_df, spark)
prelievi_aggregated_df = prelievi_aggregated_df.withColumnRenamed('sum_of_IMPORTO_by_CATEGORY_LIV0_(prelievo_contante)_and_by_SEGNO_(-)', 'prelievo_contante')
prelievi_aggregated_df = prelievi_aggregated_df.withColumn("bucket_start", col("bucket").getItem("start"))


[('CATEGORY_LIV0', ['prelievo_contante']), ('SEGNO', ['-'])]


In [16]:
##### NUOVA 
plot_in_timerange(prelievi_aggregated_df,"bucket_start","prelievo_contante","ZoomIN Totale soldi prelevati", mode="lines")

**Aggregazione 2**: 
Conteggio del numero delle spese fatte con carta:

**COUNT** di **IMPORTO** con
* **IS_CARTA** = True

In [19]:
#### AGGREGATION_2: Conteggio del numero delle spese fatte con carta:

conteggio_prelievo = [("IS_CARTA", [True])]


aggregation_step =  Aggregation(
    numerical_col_name=["IMPORTO"],
    identifier_cols_name=["ID_CLIENTE_BIC"],
    all_aggregation_filters=[conteggio_prelievo],
    agg_funcs=["count"],
)

count_prelievi_aggregated_df = aggregation_step(time_bucket_df, spark)
count_prelievi_aggregated_df = count_prelievi_aggregated_df.withColumnRenamed('count_IMPORTO_by_IS_CARTA_(True)', 'conteggio_prelievo')
count_prelievi_aggregated_df = count_prelievi_aggregated_df.withColumn("bucket_start", col("bucket").getItem("start"))


[('IS_CARTA', [True])]


In [20]:
##### NUOVA 
plot_in_timerange(count_prelievi_aggregated_df,"bucket_start","conteggio_prelievo","ZoomIN Totale soldi prelevati", mode="lines")

**Aggregazione 3**: Totale di tutti i soldi spesi con carte in determinate categorie:

**SUM** di **IMPORTO** con:
* **CATEGORY_LIV0** = shopping, salute, servizi_professionali, casa, utenze, viaggi, tempo_libero, spese_legali, tasse, trasporti, assicurazioni, scuola_formazione, altre_spese, alimentari_spesa, spese_postali
* **IS_CARTA** = True
* **SEGNO** = "-"

In [21]:
#### AGGREGATION_3: Conteggio del numero delle spese fatte con carta:

conteggio_prelievo = [
    ("CATEGORY_LIV0", ["shopping","salute","servizi_professionali","casa","utenze","viaggi","tempo_libero","spese_legali","tasse","trasporti","assicurazioni","scuola_formazione","altre_spese","alimentari_spesa","spese_postali"]),
    ("IS_CARTA", [True]),
    ("SEGNO",["-"]),                     
]


aggregation_step =  Aggregation(
    numerical_col_name=["IMPORTO"],
    identifier_cols_name=["ID_CLIENTE_BIC"],
    all_aggregation_filters=[conteggio_prelievo],
    agg_funcs=["sum"],
)

count_spese_carta_df = aggregation_step(time_bucket_df, spark)
count_spese_carta_df = count_spese_carta_df.withColumnRenamed('sum_of_IMPORTO_by_CATEGORY_LIV0_(shopping_salute_servizi_professionali_casa_utenze_viaggi_tempo_libero_spese_legali_tasse_trasporti_assicurazioni_scuola_formazione_altre_spese_alimentari_spesa_spese_postali)_and_by_IS_CARTA_(True)_and_by_SEGNO_(-)', 'count_spese_carta_df')
count_spese_carta_df = count_spese_carta_df.withColumn("bucket_start", col("bucket").getItem("start"))

[('CATEGORY_LIV0', ['shopping', 'salute', 'servizi_professionali', 'casa', 'utenze', 'viaggi', 'tempo_libero', 'spese_legali', 'tasse', 'trasporti', 'assicurazioni', 'scuola_formazione', 'altre_spese', 'alimentari_spesa', 'spese_postali']), ('IS_CARTA', [True]), ('SEGNO', ['-'])]


In [22]:
##### NUOVA 
plot_in_timerange(count_spese_carta_df,"bucket_start","count_spese_carta_df","ZoomIN Totale transizioni con carta", mode="lines")

# Soluzione problema 3: 
PROBLEMA: Assenza di eventi, ovvero vi sono dei "salti" nella linea temporale.
FILLING STEP: permette di aggiugnere tutte le transazioni mancanti

In [23]:
time_bucket_col_name="bucket"
identifier_cols_name=["ID_CLIENTE_BIC"]
time_bucket_size=1
time_bucket_granularity="day" 
new_timestamp_col_name="new_timestamp"
time_zone="Europe/Rome",

filled_ritiro_contanti_df = filled(prelievi_aggregated_df,time_bucket_col_name,identifier_cols_name,time_bucket_size,time_bucket_granularity,new_timestamp_col_name,time_zone)

In [24]:
##### PLOT AGGREGATION 
plot_in_timerange(prelievi_aggregated_df,"bucket_start","prelievo_contante","ZoomIN Totale soldi prelevati", mode="lines")

In [25]:
### PLOT FILLING
plot_in_timerange(filled_ritiro_contanti_df,new_timestamp_col_name,"prelievo_contante","Timeseries degli importi prelevati",mode="lines")

## Facciamo uno zoomout


In [26]:
## PLOT PRE FILLED
x_values = get_values(prelievi_aggregated_df, "bucket_start")
y_values = get_values(prelievi_aggregated_df, "prelievo_contante")
plot(x_values, y_values, x_name='Date', y_name='Soldi Prelevati', title='Time Series')

In [27]:
### PLOT PROCESSATO 
x_values = get_values(filled_ritiro_contanti_df, new_timestamp_col_name)
y_values = get_values(filled_ritiro_contanti_df, "prelievo_contante")
plot(x_values, y_values, x_name='Date', y_name='Soldi Prelevati', title='Time Series')

Prossime evoluzioni:
ESTENDERE IL TIMEBUCKETING
Time bucketing:
#### APPLICO IL TIME BUCKETING
time_bucket_step =  TimeBucketing(
        time_zone="Europe/Rome",
        time_column_name=DATA_COLUMN_NAME,
        time_bucket_size=1,
        time_bucket_granularity="day", -> week, day 
        time_bucket_col_name="bucket",
    )

Non c'è supporto ad hours, proposta di estendere il supporto a month

ESTENDERE L'AGGREGATION:
attualmente puoi solo filtrare in base al fatto che la variabile contenga o meno la lista di categoriche
non puoi mettere condizioni che il valore deve NON avere
non puoi mettere altro a parte isin



Filter(
  col_name: str,
  col_options: list[Union[str, bool]],
  col_options_negated: bool = False,
  col_options_exploded: bool = False,
)

Aggregation(
  agg_function: str,
  numerical_col_name: str,
  identifier_cols_name: list[str],
  filters: list[Filter],
)

Aggregator(
  aggregations: list[Aggregation]
)

Attuale metodo di configurazione è abbastanza scomodo:


filter_1 = Filter(
  col_name="CATEGORY_LIV0",
  col_options=[
    "shopping",
    "salute",
    "servizi_professionali",
    "casa",
    "utenze",
    "viaggi",
    "tempo_libero",
    "spese_legali",
    "tasse",
    "trasporti",
    "assicurazioni",
    "scuola_formazione",
    "altre_spese",
    "alimentari_spesa",
    "spese_postali",
  ],
)

filter_2 = Filter(
  col_name="ARCA_TIPO_CARTA",
  col_options=[],
)

filter_3 = Filter(
  col_name="IS_CARTA",
  col_options=[True],
)

filter_4 = Filter(
  col_name="SEGNO",
  col_options["-"],
)

agg_1 = Aggregation(
  agg_function="SUM",
  numerical_col_name="IMPORTO",
  identifier_cols_name=["ID_CLIENTE_BIC"],
  filters=[filter_1, filter_2, filter_3, filter_4],
)

aggregator_step = Aggregator(
  aggregations=[agg_3]
)

mettere i nomi alle nuove colonne
sum_of_IMPORTO_by_CATEGORY_LIV0_(shopping_salute_spese_postali)_and_by_IS_CARTA_(True)_and_by_SEGNO_(-)


creare un configurator manager che parsifica il file yaml